In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import sys
sys.path.append('../../')

titanic_df = pd.read_csv('titanic_data/data.csv')

In [2]:
import tabularmagic as tm
analyzer = tm.Analyzer(titanic_df, test_size=0.2, name='titanic')
analyzer

INFO: Initialization complete. Shapes of train, test DataFrames: (712, 12), (179, 12). 


titanic 
----------------------------------------------------------------------------------------
Train shape: (712, 12)                      Test shape: (179, 12)                       
----------------------------------------------------------------------------------------
Categorical variables:
  'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked' 
                                                                                        
Numerical variables:
  'PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare' 

In [3]:
report = analyzer.ml_classification(
    models=[
        tm.ml.LinearC('l1'),
        tm.ml.LinearC('l2'),
        tm.ml.TreeEnsembleC('random_forest')
    ],
    target='Survived',
    predictors=[
        'Pclass',
        'Sex',
        'Fare',
        'Age'
    ]
)

INFO: Fitting model LinearC(l1). 
INFO: Fitting LogisticRegression. Search method: optuna (100 trials). 
INFO: Fitted model LinearC(l1). 
INFO: Fitting model LinearC(l2). 
INFO: Fitting LogisticRegression. Search method: optuna (100 trials). 
INFO: Fitted model LinearC(l2). 
INFO: Fitting model TreeEnsembleC(random_forest). 
INFO: Fitting RandomForestClassifier. Search method: optuna (100 trials). 
INFO: Fitted model TreeEnsembleC(random_forest). 


In [4]:
report.fit_statistics()

,LinearC(l1),LinearC(l2),TreeEnsembleC(random_forest)
Statistic,,,
accuracy,0.802817,0.830986,0.823944
f1,0.75,0.777778,0.766355
precision,0.777778,0.84,0.836735
recall,0.724138,0.724138,0.706897
roc_auc,0.889368,0.899425,0.881158
n,142,142,142
